<a href="https://colab.research.google.com/github/KenyanGeek/Helicopter-vs-Truck-Logistics/blob/master/Helicopter_vs_Truck_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Comparing the cost of the most efficient logistical solution for examination distribution for Kenyan Counties using GA's**


Every year in Kenya two national exams take place - the Kenya Primary Certificate Education and Kenya Secondary School Education Final National exams. These exams determine which public schools candidates are placed in the next year depending on their performances and preferences. The exams are first distributed to county headquarters and then to the respective examination centers(mostly respective schools where candidates are based) within the counties. Our problem will look at the logistical cost of moving these exam papers by air and road to the 46 Kenyan counties and determine if it is cheaper to fly these exams or drive a trailer around the country doing the same. It is assumed that the load for the respective modes can be carried by one aircraft and truck respectively and that they will only be making one trip around the country and then will be back to Nairobi the original dispatch point.

# Code for getting absolute distances for helicopter matrix 

#### written by Herman Njoroge Chege

##### should publish this on medium or as a paper for helicopter pilot planning/campaign planning

In [0]:
#Here i am using the names to get the coordinates automatically

import pandas as pd

df = pd.DataFrame({'name': ['nairobi', 'nakuru', 'mombasa','kericho','isiolo','kisumu','garissa','mombasa','kwale','kilifi',
                            'bura','lamu','mwatate','wajir','mandera','marsabit','meru','kathwana','embu','kitui','machakos',
                            'wote','ol kalou','nyeri','kerugoya','muranga','kiambu','lodwar','kapenguria','maralal','kitale',
                            'eldoret','iten','kapsabet','kabarnet','nanyuki','narok','kajiado','bomet','kakamega','vihiga',
                            'bungoma','busia','siaya','homabay','migori','kisii','nyamira']})

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df['location'] = df['name'].apply(geocode)

In [0]:
# Here i will create two columns one for latitude and another for longitude and a tuple of both

df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df['latitude'] = df['point'].apply(lambda xy: (xy[0]))
df['longitude'] = df['point'].apply(lambda xy: (xy[1]))
df['latlong'] = df['point'].apply(lambda xy: (xy[0],xy[1]))
#Here i view my datagrame
df

,name,location,point,latitude,longitude,latlong
0,nairobi,"(Nairobi, Kenya, (-1.2832533, 36.8172449))","(-1.2832533, 36.8172449, 0.0)",-1.283253,36.817245,"(-1.2832533, 36.8172449)"
1,nakuru,"(Nakuru, Kenya, (-0.4598213, 36.10067977858))","(-0.4598213, 36.10067977858, 0.0)",-0.459821,36.100680,"(-0.4598213, 36.10067977858)"
2,mombasa,"(Mombasa, Kenya, (-4.0390146, 39.648390961857))","(-4.0390146, 39.648390961857, 0.0)",-4.039015,39.648391,"(-4.0390146, 39.648390961857)"
3,kericho,"(Kericho, Kenya, (-0.3209968, 35.2261275977538))","(-0.3209968, 35.2261275977538, 0.0)",-0.320997,35.226128,"(-0.3209968, 35.2261275977538)"
4,isiolo,"(Isiolo, Kenya, (1.00606005, 38.7478953854063))","(1.00606005, 38.7478953854063, 0.0)",1.006060,38.747895,"(1.00606005, 38.7478953854063)"
5,kisumu,"(Kisumu, 40100, Kenya, (-0.1029109, 34.7541761))","(-0.1029109, 34.7541761, 0.0)",-0.102911,34.754176,"(-0.1029109, 34.7541761)"
6,garissa,"(Garissa, Kenya, (-0.52360305, 40.3563869057871))","(-0.52360305, 40.3563869057871, 0.0)",-0.523603,40.356387,"(-0.52360305, 40.3563869057871)"
7,mombasa,"(Mombasa, Kenya, (-4.0390146, 39.648390961857))","(-4.0390146, 39.648390961857, 0.0)",-4.039015,39.648391,"(-4.0390146, 39.648390961857)"
8,kwale,"(Kwale, Kenya, (-4.1836067, 39.1050955282177))","(-4.1836067, 39.1050955282177, 0.0)",-4.183607,39.105096,"(-4.1836067, 39.1050955282177)"
9,kilifi,"(Kilifi, Kenya, (-3.15073925, 39.6750715919372))","(-3.15073925, 39.6750715919372, 0.0)",-3.150739,39.675072,"(-3.15073925, 39.6750715919372)"


In [0]:
my_list = df["latlong"].tolist()
my_list2 = df["latlong"].tolist()

# here i want to create a matrix hence i separate them into two matrices

In [0]:
#Here i calculate the absolute distance using some complicated magic

import numpy as np

earth_radius = 3958.75



def new_get_distances(loc1, loc2):
    earth_radius = 3958.75

    locs_1 = np.deg2rad(loc1)
    locs_2 = np.deg2rad(loc2)

    lat_dif = (locs_1[:,0][:,None]/2 - locs_2[:,0]/2)
    lon_dif = (locs_1[:,1][:,None]/2 - locs_2[:,1]/2)

    np.sin(lat_dif, out=lat_dif)
    np.sin(lon_dif, out=lon_dif)

    np.power(lat_dif, 2, out=lat_dif)
    np.power(lon_dif, 2, out=lon_dif)

    lon_dif *= ( np.cos(locs_1[:,0])[:,None] * np.cos(locs_2[:,0]) )
    lon_dif += lat_dif

    np.arctan2(np.power(lon_dif,.5), np.power(1-lon_dif,.5), out = lon_dif)
    lon_dif *= ( 2 * earth_radius )

    return lon_dif
locations_1 = np.asarray(my_list)
locations_2 = np.asarray(my_list2)

new = new_get_distances(locations_1,locations_2)
# Here i view my derived array

new



array([[  0.        ,  75.41543961, 272.81565665, ..., 174.16706358,
        147.79795624, 137.2085616 ],
       [ 75.41543961,   0.        , 348.03633261, ..., 129.67370116,
         93.22048532,  81.7106466 ],
       [272.81565665, 348.03633261,   0.        , ..., 423.37553379,
        409.50392111, 400.79799341],
       ...,
       [174.16706358, 129.67370116, 423.37553379, ...,   0.        ,
         39.90598692,  50.07529491],
       [147.79795624,  93.22048532, 409.50392111, ...,  39.90598692,
          0.        ,  11.53051179],
       [137.2085616 ,  81.7106466 , 400.79799341, ...,  50.07529491,
         11.53051179,   0.        ]])

In [0]:
#Here i convert the derived distances into a dataframe
distances = pd.DataFrame(new,index=new[:,0])

distances

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0.000000,0.000000,75.415440,272.815657,128.465764,206.909100,164.213638,250.068751,272.815657,255.114797,235.745905,4059.088025,287.904461,207.932661,314.120842,411.458541,292.778988,123.553096,109.352044,78.092235,109.185492,39.928696,65.751218,76.541895,67.239385,63.023751,33.834353,17.354917,315.411543,211.918069,164.865761,202.587834,163.173288,162.623429,156.737652,143.626903,91.380354,92.549947,57.971682,115.794075,185.746740,173.629601,203.424923,210.154340,199.503706,171.067692,174.167064,147.797956,137.208562
75.415440,75.415440,0.000000,348.036333,61.180785,209.068566,96.245831,294.062691,348.036333,330.458418,309.011888,4010.942581,352.339422,282.879131,317.738345,407.531036,254.928702,125.142339,112.823645,108.228635,174.527255,105.841483,139.537223,23.444177,58.213989,81.421447,67.525966,63.272965,249.514672,137.310492,115.453798,127.309038,87.765177,88.370146,82.769663,70.543765,74.834633,71.057289,124.207235,62.148961,111.425383,103.275583,128.365153,139.236393,134.103957,113.562120,129.673701,93.220485,81.710647
272.815657,272.815657,348.036333,0.000000,398.978954,354.082221,433.724437,247.758935,0.000000,38.751347,61.401375,4220.282642,147.395238,82.050367,413.886273,507.416320,494.870685,321.719970,315.388791,278.064984,192.317103,246.056251,209.210741,344.822926,318.325044,294.405321,287.096292,285.283318,567.953070,482.373040,409.291622,474.436750,435.562001,433.186749,429.369778,413.403555,331.650816,345.325601,237.779314,381.133874,458.421175,444.347701,476.239616,480.628119,465.805045,432.586689,423.375534,409.503921,400.797993
128.465764,128.465764,61.180785,398.978954,0.000000,260.023142,35.921589,354.732646,398.978954,378.042744,364.156954,4011.822348,412.379752,328.705781,366.430687,450.935863,278.935143,180.263671,169.383700,169.178844,233.736970,163.824236,194.216192,79.748663,117.794106,142.323675,127.842429,120.308510,238.914614,109.028868,141.452574,93.827953,57.851661,71.726601,37.052001,66.832680,129.799248,68.359399,164.599786,27.476535,63.598585,45.398459,83.903322,81.848252,73.062519,54.961864,79.683649,39.777902,30.549644
206.909100,206.909100,209.068566,354.082221,260.023142,0.000000,286.367201,153.363836,354.082221,359.417915,294.258920,3882.863977,270.305366,331.857862,109.241381,204.571307,146.472497,86.057978,100.081781,130.177058,179.469699,183.466429,207.457382,185.764315,155.277836,145.192552,174.962513,194.040523,261.731980,253.284459,141.620589,258.898782,241.620162,224.597417,258.023988,210.553879,134.397277,275.604258,255.081300,270.122030,274.912274,286.303430,278.745059,312.818507,322.651237,314.780603,337.113004,298.360695,287.360420
164.213638,164.213638,96.245831,433.724437,35.921589,286.367201,0.000000,388.157996,433.724437,411.948962,399.781890,4002.815721,448.146710,362.249419,390.467590,471.060728,289.277883,210.102895,200.220809,203.348803,269.609425,199.741713,229.943265,112.829016,151.127112,176.522222,163.433865,156.228092,229.988549,96.378032,158.052908,79.296365,56.256782,75.190000,31.846461,79.902042,160.531163,95.330689,197.888334,54.142765,41.489982,13.242948,61.224793,47.078637,38.313676,35.873229,70.487636,39.152687,39.969366
250.068751,250.068751,294.062691,247.758935,354.732646,153.363836,388.157996,0.000000,247.758935,267.223624,187.514938,3973.258996,126.629231,260.047167,171.717177,260.366658,296.412151,185.561619,191.543761,185.929739,154.744820,211.119641,207.446234,275.377762,237.095222,212.655241,232.522746,247.125024,413.724321,383.985514,276.407595,385.084618,357.633426,344.715590,366.527973,326.440079,229.766624,341.064738,270.190625,353.803418,390.204909,392.503228,399.795739,425.399680,426.471628,407.487189,419.161838,386.244222,374.746435
272.815657,272.815657,348.036333,0.000000,398.978954,354.082221,433.724437,247.758935,0.000000,38.751347,61.401375,4220.282642,147.395238,82.050367,413.886273,507.416320,494.8

In [0]:
distances.shape

(48, 48)